In [ ]:
import yaml
from pathlib import Path
structure_file = "../data/structure.yaml"

In [ ]:
def criar_estrutura(livro: list, base_dir: str = "."):
    """
    Cria uma estrutura de diretórios e arquivos markdown a partir de uma lista de capítulos (YAML carregado).
    
    Args:
        livro: Lista contendo os capítulos e seções.
        base_dir: Diretório base onde a estrutura será criada.
    """
    
    def criar_stub(caminho_arquivo: str, titulo: str, nivel: int):
        """Cria um arquivo markdown stub com um título no nível adequado."""
        with open(caminho_arquivo, "w", encoding="utf-8") as f:
            f.write(f"{'#' * nivel} {titulo}\n\n")
            f.write("*este é um stub*\n")
    
    def processar_nivel(nivel: int, itens: list, prefixo: str = "CAP", caminho_atual: str = base_dir):
        
        for i, item in enumerate(itens, start=1):
            if isinstance(item, dict):
                for titulo, subitens in item.items():
                    id_pasta = f"{prefixo}_{i:02}"
                    caminho_pasta = Path(caminho_atual) / id_pasta
                    Path(caminho_pasta).mkdir(exist_ok=True)
                    quadro_cap = Path(caminho_pasta) / f"{caminho_pasta.name}_QUADROS"
                    Path(quadro_cap).mkdir(exist_ok=True)
                    caminho_md = Path(caminho_pasta) / f"{id_pasta}.md"
                    criar_stub(caminho_md, titulo, nivel)

                    if subitens:  # Se houver subseções
                        processar_nivel(nivel + 1, subitens, prefixo=f"SEC_{i:02}", caminho_atual=caminho_pasta)
            else:
                # Caso seja apenas uma string (seção sem subseções)
                id_pasta = f"{prefixo}_{i:02}"
                caminho_pasta = Path(caminho_atual) / id_pasta
                Path(caminho_pasta).mkdir(exist_ok=True)

                caminho_md = Path(caminho_pasta) / f"{id_pasta}.md"
                criar_stub(caminho_md, item, nivel)
    
    processar_nivel(1, livro)



In [63]:

def gerar_toc_yaml(base_dir: Path, root: str, toc_filename: Path = Path("_toc.yaml")):
    """
    Gera um arquivo _toc.yaml compatível com Jupyter Book.

    Args:
        base_dir (Path): Diretório base onde está a estrutura de capítulos.
        root (str): Caminho (sem extensão) para o arquivo raiz do livro (ex: "CAP_01/CAP_01").
        toc_filename (Path): Caminho para o arquivo _toc.yaml a ser gerado.
    """

    def processar_pasta(pasta: Path):
        entradas = []
        for subpasta in sorted(pasta.iterdir()):
            if subpasta.is_dir():
                files = sorted([file for file in subpasta.glob("*.*") if file.suffix in {".md", ".ipynb"}])
                for file in files:
                    rel_path = file.relative_to(base_dir).name
                    entry = {"file": rel_path}
                
                # Recursivamente processa subpastas
                subpastas = [p for p in subpasta.iterdir() if p.is_dir()]
                if subpastas and len(files) > 0:
                    entry["sections"] = processar_pasta(subpasta)
                    entradas.append(entry)
        return entradas

    toc = {
        "format": "jb-book",
        "root": root,
        "sections": processar_pasta(base_dir)
    }

    toc_filename = Path(toc_filename)
    with toc_filename.open("w", encoding="utf-8") as f:
        yaml.dump(toc, f, allow_unicode=True, sort_keys=False)

    print(f"Arquivo '{toc_filename}' gerado com sucesso.")



In [64]:
with open(structure_file) as file:
    estrutura = yaml.safe_load(file)
criar_estrutura(estrutura['livro'], base_dir="./content/")

In [65]:
gerar_toc_yaml(
    base_dir=Path("./content/"),
    root="CAP_01/CAP_01",
    toc_filename=Path("_toc.yaml")
)

Arquivo '_toc.yaml' gerado com sucesso.


In [30]:
base_dir=Path("./")
root="CAP_01/CAP_01"
def processar_pasta(pasta: Path):
    entradas = []
    entry = {}
    for subpasta in sorted(pasta.iterdir()):
        if subpasta.is_dir():
            files = sorted([file for file in subpasta.glob("*.*") if file.suffix in {".md", ".ipynb"}])
            for file in files:
                rel_path = file.relative_to(base_dir).name
                entry = {"file": rel_path}

            # Recursivamente processa subpastas
            subpastas = [p for p in subpasta.iterdir() if p.is_dir()]
            if subpastas:
                entry["sections"] = processar_pasta(subpasta)

            entradas.append(entry)
    return entradas

In [31]:
base_dir

PosixPath('.')

In [32]:
processar_pasta(base_dir)

[{'file': 'CAP_01.md', 'sections': [{}]},
 {'file': 'CAP_02.md', 'sections': [{}]},
 {'file': 'CAP_03.md',
  'sections': [{},
   {'file': 'SEC_03_01.md', 'sections': [{}]},
   {'file': 'SEC_03_02.md', 'sections': [{}]},
   {'file': 'SEC_03_03.md', 'sections': [{}]},
   {'file': 'SEC_03_04.md', 'sections': [{}]},
   {'file': 'SEC_03_05.md',
    'sections': [{},
     {'file': 'SEC_05_01.md', 'sections': [{}]},
     {'file': 'SEC_05_02.md', 'sections': [{}]},
     {'file': 'SEC_05_03.md', 'sections': [{}]},
     {'file': 'SEC_05_04.md', 'sections': [{}]}]},
   {'file': 'SEC_03_06.md', 'sections': [{}]}]},
 {'file': 'CAP_04.md',
  'sections': [{},
   {'file': 'SEC_04_01.md', 'sections': [{}]},
   {'file': 'SEC_04_02.md',
    'sections': [{'file': 'SEC_02_01.md', 'sections': [{}]},
     {'file': 'SEC_02_02.md', 'sections': [{}]},
     {'file': 'SEC_02_02.md', 'sections': [{}]}]},
   {'file': 'SEC_04_03.md', 'sections': [{}]},
   {'file': 'SEC_04_04.md',
    'sections': [{'file': 'SEC_04_01.

In [6]:
from pathlib import Path
base_dir = Path("../content/")
root = "index.md"
files = sorted([file.relative_to(base_dir) for file in base_dir.rglob("**/*.md") if (file.suffix in [".md", ".ipynb"]) and (file.name != root) and ("QUADROS" not in str(file))])

In [7]:
files

[PosixPath('CAP_01/CAP_01.md'),
 PosixPath('CAP_02/CAP_02.md'),
 PosixPath('CAP_03/CAP_03.md'),
 PosixPath('CAP_03/SEC_03_01/SEC_03_01.md'),
 PosixPath('CAP_03/SEC_03_02/SEC_03_02.md'),
 PosixPath('CAP_03/SEC_03_03/SEC_03_03.md'),
 PosixPath('CAP_03/SEC_03_04/SEC_03_04.md'),
 PosixPath('CAP_03/SEC_03_05/SEC_03_05.md'),
 PosixPath('CAP_03/SEC_03_05/SEC_05_01/SEC_05_01.md'),
 PosixPath('CAP_03/SEC_03_05/SEC_05_02/SEC_05_02.md'),
 PosixPath('CAP_03/SEC_03_05/SEC_05_03/SEC_05_03.md'),
 PosixPath('CAP_03/SEC_03_05/SEC_05_04/SEC_05_04.md'),
 PosixPath('CAP_03/SEC_03_06/SEC_03_06.md'),
 PosixPath('CAP_04/CAP_04.md'),
 PosixPath('CAP_04/SEC_04_02/SEC_02_01/SEC_02_01.md'),
 PosixPath('CAP_04/SEC_04_02/SEC_02_02/03-distribuicoes.md'),
 PosixPath('CAP_04/SEC_04_02/SEC_02_02/SEC_02_02.md'),
 PosixPath('CAP_04/SEC_04_02/SEC_04_02.md'),
 PosixPath('CAP_04/SEC_04_03/03-01-medidas.md'),
 PosixPath('CAP_04/SEC_04_03/SEC_04_03.md'),
 PosixPath('CAP_04/SEC_04_04/04-integral-gaussiana.md'),
 PosixPath('CA